<a href="https://colab.research.google.com/github/DobTurMisp/MISP/blob/master/0_Zbiorowe_sprawozdanie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sprawozdanie z laboratoruim Metod i Systemów Sterownia Produkcją**   
Mateusz Dobrowolski 208888
Konrad Turek 208999


**Zadanie domowe: Książka telefoniczna**

Zadanie polegało na stworzeniu programu który będzie rozpoznawał poprawność wpisanego numeru telefonu, oraz da odpowiedź na pytanie czy numer jest komórkowy czy stacjonarny. W przypadku numeru stacjonarnego, program podawał z jakiego miasta on pochodzi na podstawie wprowadzonych kodów regionalnych

In [0]:
#Utworzenie klasy NumerTelefonu
class NumerTelefonu():
  #Definiowanie słów odpowiadających cyfrom oraz definiowanie numerów kierunkowych do województw
  def __init__ (self, numer):
    self.numer = numer
    self.podawanie_numeru = {"0":'zero', "1":'jeden', "2":'dwa', "3":'trzy', "4":'cztery', "5":'pięć', "6":'sześć', "7":'siedem', "8":'osiem', "9":'dziewięć'}
    self.kody_regionalne = {"42":"Łódź", "41":"Koluszki", "22":"Warszawa", "62":"Koło"}
  #Sprawdzanie długości numeru (Jak inna niż 9 cyfr to numer niepoprawny)
  def spr(self):
      if len(self.numer)!=9:
         print ("error")
      else:
        print ("Podany numer telefonu to: {}".format(self.numer))
  #Sprawdzanie czy dany numer jest numerem stacjonarnym czy komórkowym. Jeżeli pierwsze dwie cyfry numeru znajdują się w słowniku kody_regionalne
  #Uznajemy, że jest to numer stacjonarny. W przeciwnym wypadku, mamy doczynienia z numerem komórkowym.
  def wojewodztwo(self):
    if len(self.numer)!=9:
       print ("error")
    else:
      if self.numer[:2] in self.kody_regionalne:
         print(self.kody_regionalne[self.numer[:2]])
      else:
         print ("Jest to numer komórkowy")
  #Podawanie kolejnych słów ze słownika podawanie_numeru na podstawie przeszukiwania klucza w podanym numerze w funkcji for.
  def podawanie (self):
      podawanie_slowne=""
      for numery in self.numer:
          podawanie_slowne+= (" "+self.podawanie_numeru[numery])
      return podawanie_slowne
          

In [0]:
#Wywołanie funkcji
numer=NumerTelefonu("426727582")
numer.spr()
numer.wojewodztwo()
numer.podawanie()

**Zadanie: Prąd maksymalny**

Zadanie polegało na użyciu funkcji optymalizacyjnej obliczającą maksymalną wartość zdefiniowanego problemu. Na podstawie schematu elektrycznego musieliśmy obliczyć prąd maksymalny który będzie mógł przepłynąć przez konkretną gałąź obwodu elektrycznego. Potrzebna do tego była znajomość podstawowych praw elektrotechniki.

In [0]:
#Definiowanie problemu i zmiennych decyzyjnych
import pulp

prad_max = pulp.LpProblem("prad_max", pulp.LpMaximize)

I_1 = pulp.LpVariable('I_1', lowBound=0, cat='Continuous')
I_2 = pulp.LpVariable('I_2', lowBound=0, cat='Continuous')
I_3 = pulp.LpVariable('I_3', lowBound=0, cat='Continuous')
I_4 = pulp.LpVariable('I_4', lowBound=0, cat='Continuous')
I_5 = pulp.LpVariable('I_5', lowBound=0, cat='Continuous')

In [0]:
# Funkcja celu
prad_max += I_1 + I_2, "Rozwiazanie"

# Ograniczenia

prad_max += I_1 <=2, "I1"
prad_max += I_2 <=3, "I2"
prad_max += I_3 <=4, "I3"
prad_max += I_4 <=2, "I4"
prad_max += I_5 <=2, "I5"
prad_max += I_1 * 8 == I_2 * 6, "K1"
prad_max += I_4 * 10 == I_5 * 8,"K2"
prad_max += I_1+I_2 == I_3, "KK1"
prad_max += I_4+I_5 == I_3, "KK2"

In [0]:
prad_max

Etap rozwiązania zadania

In [0]:
#Rozwiązywanie funkcji
prad_max.solve()

In [0]:
#Podanie wartości obliczonych zmiennych
for variable in prad_max.variables():
    print("{} = {}".format(variable.name, variable.varValue))

In [0]:
#Wynik rozwiązania problemu
print(pulp.value(prad_max.objective))

In [0]:
print("""\nSensitivity Analysis\n
        Name\tConstraint\t\t\tShadow Price\t\tSlack""")
for name, c in prad_max.constraints.items():
    print(str(name).ljust(17), ":", str(c).ljust(33), str(c.pi).ljust(17), str(c.slack).ljust(10))

**Zadanie: Scheduling problem**

Scheduling problem jest zadaniem optymalizacyjnym które ma dać nam odpowiedź na pytania dotyczące układania grafików. W naszym zadaniu mieliśmy stworzyć grafik dla pracowników biblioteki w jak najbardziej optymalny sposób, oraz dodatkowo dobrać czas pracy konkretnych pracowników w taki sposób aby biblioteka wydała najmniej pięniędzy na pensję dla nich.

In [0]:
from pulp import *
import pandas as pd
#Utworzenie problemu funkcji minimalizacji i stworzenie zmiennych decyzyjnych

plan = pulp.LpProblem("plan", pulp.LpMinimize)
#Grafik na dni tygodnia dla każdego pracownika
gpIgorPon = pulp.LpVariable('Godziny pracy Igor Pon', lowBound=0, cat='Continous')
gpIgorWt = pulp.LpVariable('Godziny pracy Igor Wt', lowBound=0, cat='Continous')
gpIgorSr = pulp.LpVariable('Godziny pracy Igor Sr', lowBound=0, cat='Continous')
gpIgorCzw = pulp.LpVariable('Godziny pracy Igor Czw', lowBound=0, cat='Continous')
gpIgorPt = pulp.LpVariable('Godziny pracy Igor Pt', lowBound=0, cat='Continous')
gpIgorSob = pulp.LpVariable('Godziny pracy Igor Sob', lowBound=0, cat='Continous')

gpMarcinPon = pulp.LpVariable('Godziny pracy Marcin Pon', lowBound=0, cat='Continous')
gpMarcinWt = pulp.LpVariable('Godziny pracy Marcin Wt', lowBound=0, cat='Continous')
gpMarcinSr = pulp.LpVariable('Godziny pracy Marcin Sr', lowBound=0, cat='Continous')
gpMarcinCzw = pulp.LpVariable('Godziny pracy Marcin Czw', lowBound=0, cat='Continous')
gpMarcinPt = pulp.LpVariable('Godziny pracy Marcin Pt', lowBound=0, cat='Continous')
gpMarcinSob = pulp.LpVariable('Godziny pracy Marcin Sob', lowBound=0, cat='Continous')

gpFranekPon = pulp.LpVariable('Godziny pracy Franek Pon', lowBound=0, cat='Continous')
gpFranekWt = pulp.LpVariable('Godziny pracy Franek Wt', lowBound=0, cat='Continous')
gpFranekSr = pulp.LpVariable('Godziny pracy Franek Sr', lowBound=0, cat='Continous')
gpFranekCzw = pulp.LpVariable('Godziny pracy Franek Czw', lowBound=0, cat='Continous')
gpFranekPt = pulp.LpVariable('Godziny pracy Franek Pt', lowBound=0, cat='Continous')
gpFranekSob = pulp.LpVariable('Godziny pracy Franek Sob', lowBound=0, cat='Continous')


gpPiotrPon = pulp.LpVariable('Godziny pracy Piotr Pon', lowBound=0, cat='Continous')
gpPiotrWt = pulp.LpVariable('Godziny pracy Piotr Wt', lowBound=0, cat='Continous')
gpPiotrSr = pulp.LpVariable('Godziny pracy Piotr Sr', lowBound=0, cat='Continous')
gpPiotrCzw = pulp.LpVariable('Godziny pracy Piotr Czw', lowBound=0, cat='Continous')
gpPiotrPt = pulp.LpVariable('Godziny pracy Piotr Pt', lowBound=0, cat='Continous')
gpPiotrSob = pulp.LpVariable('Godziny pracy Piotr Sob', lowBound=0, cat='Continous')

gpAdamPon = pulp.LpVariable('Godziny pracy Adam Pon', lowBound=0, cat='Continous')
gpAdamWt = pulp.LpVariable('Godziny pracy Adam Wt', lowBound=0, cat='Continous')
gpAdamSr = pulp.LpVariable('Godziny pracy Adam Sr', lowBound=0, cat='Continous')
gpAdamCzw = pulp.LpVariable('Godziny pracy Adam Czw', lowBound=0, cat='Continous')
gpAdamPt = pulp.LpVariable('Godziny pracy Adam Pt', lowBound=0, cat='Continous')
gpAdamSob = pulp.LpVariable('Godziny pracy Adam Sob', lowBound=0, cat='Continous')

gpZuzaPon = pulp.LpVariable('Godziny pracy Zuza Pon', lowBound=0, cat='Continous')
gpZuzaWt = pulp.LpVariable('Godziny pracy Zuza Wt', lowBound=0, cat='Continous')
gpZuzaSr = pulp.LpVariable('Godziny pracy Zuza Sr', lowBound=0, cat='Continous')
gpZuzaCzw = pulp.LpVariable('Godziny pracy Zuza Czw', lowBound=0, cat='Continous')
gpZuzaPt = pulp.LpVariable('Godziny pracy Zuza Pt', lowBound=0, cat='Continous')
gpZuzaSob = pulp.LpVariable('Godziny pracy Zuza Sob', lowBound=0, cat='Continous')


#Godziny czasu pracy
gpIgor = pulp.LpVariable('Godziny pracy Igor', lowBound=0, cat='Continous')
gpMarcin = pulp.LpVariable('Godziny pracy Marcin', lowBound=0, cat='Continous')
gpFranek = pulp.LpVariable('Godziny pracy Franek', lowBound=0, cat='Continous')
gpPiotr = pulp.LpVariable('Godziny pracy Piotr', lowBound=0, cat='Continous')
gpAdam = pulp.LpVariable('Godziny pracy Adam', lowBound=0, cat='Continous')
gpZuza = pulp.LpVariable('Godziny pracy Zuza', lowBound=0, cat='Continous')

#Dostępność biblioteki
dPon = pulp.LpVariable('Dostepnosc pon', lowBound=0, cat='Continous')
dWt = pulp.LpVariable('Dostepnosc wt', lowBound=0, cat='Continous')
dSr = pulp.LpVariable('Dostepnosc sr', lowBound=0, cat='Continous')
dCzw = pulp.LpVariable('Dostepnosc czw', lowBound=0, cat='Continous')
dPt = pulp.LpVariable('Dostepnosc pt', lowBound=0, cat='Continous')
dSob = pulp.LpVariable('Dostepnosc sob', lowBound=0, cat='Continous')
dNd = pulp.LpVariable('Dostepnosc nd', lowBound=0, cat='Continous')

#Dostępność pracowników
gdIgor = pulp.LpVariable('Dostępność Igor', lowBound=0, cat='Continous')
gdMarcin = pulp.LpVariable('Dostępność Marcin', lowBound=0, cat='Continous')
gdFranek = pulp.LpVariable('Dostępność Franek', lowBound=0, cat='Continous')
gdPiotr = pulp.LpVariable('Dostępność Piotr', lowBound=0, cat='Continous')
gdAdam = pulp.LpVariable('Dostępność Adam', lowBound=0, cat='Continous')
gdZuza = pulp.LpVariable('Dostępność Zuza', lowBound=0, cat='Continous')

In [0]:
#Główny problem
plan += dPon + dWt + dSr + dCzw + dPt + dSob + dNd == gpIgor + gpMarcin + gpFranek + gpPiotr + gpAdam + gpZuza, "Godziny pracy"
plan += (gpIgor * 170) + (gpMarcin * 60) + (gpFranek * 80) + (gpPiotr * 200) + (gpAdam * 90) +(gpZuza * 10), "Koszt pracy"


#Grafik na dni tyg
plan += gpIgorPon+gpIgorWt+gpIgorSr+gpIgorCzw+gpIgorPt+gpIgorSob==gpIgor
plan += gpPiotrPon+gpPiotrWt+gpPiotrSr+gpPiotrCzw+gpPiotrPt+gpPiotrSob==gpPiotr
plan += gpZuzaPon+gpZuzaWt+gpZuzaSr+gpZuzaCzw+gpZuzaPt+gpZuzaSob==gpZuza
plan += gpFranekPon+gpFranekWt+gpFranekSr+gpFranekCzw+gpFranekPt+gpFranekSob==gpFranek
plan += gpAdamPon+gpAdamWt+gpAdamSr+gpAdamCzw+gpAdamPt+gpAdamSob==gpAdam
plan += gpMarcinPon+gpMarcinWt+gpMarcinSr+gpMarcinCzw+gpMarcinPt+gpMarcinSob==gpMarcin


plan += gpIgorPon + gpPiotrPon + gpZuzaPon + gpMarcinPon + gpAdamPon + gpFranekPon<=8
plan += gpIgorWt + gpPiotrWt + gpZuzaWt + gpMarcinWt + gpAdamWt + gpFranekWt <=4
plan += gpIgorSr + gpPiotrSr + gpZuzaSr + gpMarcinSr + gpAdamSr + gpFranekSr <=4
plan += gpIgorCzw + gpPiotrCzw + gpZuzaCzw + gpMarcinCzw + gpAdamCzw + gpFranekCzw<=4
plan += gpIgorPt + gpPiotrPt + gpZuzaPt + gpMarcinPt + gpAdamPt + gpFranekPt <=8
plan += gpIgorSob + gpPiotrSob + gpZuzaSob + gpMarcinSob + gpAdamSob + gpFranekSob <=2

#Ograniczenia

#Maksymalna dostępność danych pracowników w ciągu tygodnia
plan += gdIgor == 4, "Czas Igora"
plan += gdMarcin == 4, "Czas Marcina"
plan += gdFranek == 3, "Czas Franka"
plan += gdPiotr == 8, "Czas Piotra"
plan += gdAdam == 10, "Czas Adama"
plan += gdZuza == 5, "Czas Zuzy"


#Ilość godzin otwarcia biblioteki z podziałem na dni
plan += dPon == 8, "Otwarte w pon"
plan += dWt == 4, "Otwarte we wt"
plan += dSr == 4, "Otwarte w sr"
plan += dCzw == 4, "Otwarte w czw"
plan += dPt == 8, "Otwarte w pt"
plan += dSob == 2, "Otwarte w sob"
plan += dNd == 0, "Otwarte w nd"

#Ograniczenie mówiące, że nie mogą pracować dłużej niż są dostępni
plan += gpIgor <= gdIgor, "Czas pracy Igora"
plan += gpMarcin <= gdMarcin, "Czas pracy Marcina"
plan += gpFranek <= gdFranek, "Czas pracy Franka"
plan += gpPiotr <= gdPiotr, "Czas pracy Piotra"
plan += gpAdam <= gdAdam, "Czas pracy Adama"
plan += gpZuza <= gdZuza, "Czas pracy Zuzy"


Etap rozwiązywania problemu

In [0]:
plan.solve()

In [0]:
#Podanie godzin pracy poszczególnych pracowników w konkretne dni
for variable in plan.variables():
    print("{} = {}".format(variable.name, variable.varValue))

In [0]:
#Wynik rozwiązania minimalizacji pensji dla pracowników
print('Pensja tygodniowa dla pracowników:')
print(pulp.value(plan.objective))

**Zadanie: Titanic**

W tym zadaniu wykorzystane zostały mechanizmy uczenia maszynowego w sposób nadzorowany. Zadaniem programu było ustalenie kto przeżyje katastrofę statku.

In [0]:
#Zaimportowanie odpowiednich bibliotek i podanie ścieżki dostępu do zbioru testowego
import pandas as pd

test = pd.read_csv("/home/misp/nazwa_nowego_folderu2/tytaanik/test.csv")
test_shape = test.shape
print(test_shape)

In [0]:
#Scieżka do zbioru do trenowania maszyny
train = pd.read_csv("/home/misp/nazwa_nowego_folderu2/tytaanik/train.csv")
train_shape = train.shape
print(train_shape)

In [0]:
#Określenie wielkości bazy na jakiej ma się uczyć program.
#W przypadku określania ilości przykładów, trzeba uważać aby nie dostarczyć programowi za mało danych oraz za dużo.
#Gdy podamy za mało, program może nie być w stanie się odpowiednio nauczyć na zbyt okrojonej bazie
#Gdy podamy za dużo przykładów, może dojść do efektu, że program zacznie uczyć się "na pamięć"

train.head(30)

In [0]:
#Wykreślenie tabeli w której uzależniamy od siebie dane o płci pasażerów oraz o ich przeżyciu.
#Dzięki temu wiemy, że większość pasażerów która przeżyła była płci żeńskiej
import matplotlib.pyplot as plt

sex_pivot = train.pivot_table(index="Sex",values="Survived")
sex_pivot


In [0]:
#Tabela osób które przeżyły zależnie od wieku pasażerów
import matplotlib.pyplot as plt

Age_pivot = train.pivot_table(index="Age",values="Survived")
Age_pivot


In [0]:
# Z której klasy ludziki przeżyły :)
#Wykres słupkowy przedstawiający informację na temat przeżywalności katastrofy pasażerów z poszczególnych klas. 

pclass_pivot = train.pivot_table(index="Pclass",values="Survived")
pclass_pivot.plot.bar()
plt.show()

In [0]:
#Opis danej cechy 
train['Name'].describe()

In [0]:
#Wszyscy którzy przeżyli

train[train["Survived"] == 1]

In [0]:
#Wszyscy którzy umarli (nakarmili rybki)

train[train["Survived"] == 0]

In [0]:
#Wykres przedstawiający przeżywalność w zależności od wieku

survived = train[train["Survived"] == 1]
died = train[train["Survived"] == 0]
survived["Age"].plot.hist(alpha=0.5,color='red',bins=50)
died["Age"].plot.hist(alpha=0.5,color='blue',bins=50)
plt.legend(['Survived','Died'])
plt.show()

In [0]:
#Wykres przedstawiający która grupa wiekowa (zdefiniowana przez nas) cieszyła się największym "szczęściem"
def process_age(df,cut_points,label_names):
    df["Age"] = df["Age"].fillna(-0.5)
    df["Age_categories"] = pd.cut(df["Age"],cut_points,labels=label_names)
    return df
#Podział na kategorie wiekowe
cut_points = [-1,0, 5, 12, 18, 35, 60, 100]
#Przypisanie etykiet (nazw) poszczególnym grupom (na podstawie cut_points)
label_names = ["Missing", 'Infant', "Child", 'Teenager', "Young Adult", 'Adult', 'Senior']

train = process_age(train,cut_points,label_names)
test = process_age(test,cut_points,label_names)

age_cat_pivot = train.pivot_table(index="Age_categories",values="Survived")
age_cat_pivot.plot.bar()
plt.show()

In [0]:
#Ile było pasażerów w danej klasie
train['Pclass'].value_counts()

In [0]:
column_name = "Pclass"
df = train
dummies = pd.get_dummies(df[column_name],prefix=column_name)
dummies.head()

In [0]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

train = create_dummies(train,"Pclass")
test = create_dummies(test,"Pclass")
train.head()

In [0]:
train = create_dummies(train,"Sex")
test = create_dummies(test,"Sex")
train = create_dummies(train,"Age_categories")
test = create_dummies(test,"Age_categories")

In [0]:
train.head()

In [0]:
columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior']

#Zastosowanie regresji liniowej do nauki
from sklearn.linear_model import LogisticRegression


In [0]:

lr = LogisticRegression()
columns = ['Pclass_2', 'Pclass_3', 'Sex_male']
lr.fit(train[columns], train['Survived'])

In [0]:
lr.decision_function(train[columns])

In [0]:
lr.coef_

In [0]:
lr =LogisticRegression()
lr.fit(train[columns], train['Survived'])

In [0]:
holdout = test 

from sklearn.model_selection import train_test_split

columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior']

all_X = train[columns]
all_y = train['Survived']

train_X, test_X, train_y, test_y = train_test_split(
    all_X, all_y, test_size=0.2,random_state=0)

In [0]:
train_X.shape

In [0]:
from sklearn.metrics import accuracy_score
lr =LogisticRegression()
lr.fit(train_X, train_y)
predictions = lr.predict(test_X)
accuracy = accuracy_score(test_y, predictions)
accuracy

In [0]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(test_y, predictions)
pd.DataFrame(conf_matrix, columns=['Survived', 'Died'], index=[['Survived', 'Died']])



In [0]:
from sklearn.model_selection import cross_val_score
import numpy as npa

lr = LogisticRegression()
scores = cross_val_score(lr, all_X, all_y, cv=10)
np.mean(scores)

In [0]:
columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior']

In [0]:
holdout.head()

In [0]:
#Wyniki przewidywania
lr = LogisticRegression()
lr.fit(all_X, all_y)
holdout_predictions = lr.predict(holdout[columns])
holdout_predictions

In [0]:
holdout_ids = holdout["PassengerId"]
submission_df = {"PassengerId": holdout_ids,
                 "Survived": holdout_predictions}
submission = pd.DataFrame(submission_df)

submission.to_csv('titanic_submission.csv', index=False)

**Zadanie: Projekt**

Konrad Turek 208999, Mateusz Dobrowolski 208888

# **Ghouls, Goblins, and Ghosts... Boo!**

https://www.kaggle.com/c/ghouls-goblins-and-ghosts-boo/overview

Wzieliśmy udział w konkursie dotyczącym klasyfikacji potworów, aby je rozróżnić podane mieliśmy takie dane jak długość kości, kolor czy stopień zgnilizny.

In [0]:
# zaimportowanie bibliotek 
import numpy as np 
import pandas as pd 
from subprocess import check_output

In [0]:
# odczytanie plików csv 
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

In [0]:
train_outcome = train_df["type"]
train_df.drop(["type"], axis=1, inplace=True)

In [0]:
#enkodowanie danych po kolorze 
dane = pd.concat([train_df[["id", "color"]], test_df[["id", "color"]]])
dane_encoded = pd.get_dummies(dane, columns=["color"])
train_df = train_df.merge(dane_encoded, on="id", how="left")
test_df = test_df.merge(dane_encoded, on="id", how="left")
train_df.drop(["color"], axis=1, inplace=True)
test_df.drop(["color"], axis=1, inplace=True)

In [0]:
#oddzielenie identyfikatorów 
train_id = train_df[["id"]]
test_id = test_df[["id"]]
train_df.drop(["id"], axis=1, inplace=True)
test_df.drop(["id"], axis=1, inplace=True)

In [0]:
#zaimportowanie modelu estymatora
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [0]:
#uczenie na części danych
X_train, X_val, y_train, y_val = train_test_split(train_df, train_outcome, test_size=0.2)
forest = RandomForestClassifier(n_estimators=200, n_jobs=4)
forest.fit(X_train, y_train)
y_pred_val = forest.predict(X_val)

In [0]:
#wyświetlenie danych
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_val, y_pred_val))
print("Wynik: {:.1%}".format(accuracy_score(y_val, y_pred_val)))

In [0]:
#wykorzystanie pełnego zestawu danych do uczenia 
forest = RandomForestClassifier(n_estimators=500, n_jobs=4)
forest.fit(train_df, train_outcome)
y_pred = forest.predict(test_df)

In [0]:
wynik = pd.read_csv("/content/sample_submission.csv")
wynik["type"] = y_pred

In [0]:
#zapisanie wyniku do pliku csv 
wynik.to_csv("/content/submission.csv", index=False)

Wysyłając rozwiązanie na konkurs uzyskaliśmy wynik 0.71266